<a href="https://colab.research.google.com/github/Andrei198203/Data_Science_HW10_part2/blob/main/HW10_part2_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img, ImageDataGenerator

In [6]:
# Downloading data
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

In [7]:
# Data conversion and normalisation
train_images = tf.reshape(train_images, (train_images.shape[0], 28, 28, 1)).numpy().astype('float32') / 255
test_images = tf.reshape(test_images, (test_images.shape[0], 28, 28, 1)).numpy().astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [8]:
# Convert images up to 3 channels and resize
train_images_rgb = np.asarray([img_to_array(array_to_img(image).convert('RGB')) for image in train_images])
test_images_rgb = np.asarray([img_to_array(array_to_img(image).convert('RGB')) for image in test_images])

train_images_resized = tf.image.resize(train_images_rgb, (48, 48))
test_images_resized = tf.image.resize(test_images_rgb, (48, 48))

In [15]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array, array_to_img

In [16]:
# Pre-treatment in accordance with VGG16
train_images_preprocessed = preprocess_input(train_images_resized)
test_images_preprocessed = preprocess_input(test_images_resized)

In [17]:
# Loading the pre-trained VGG16 model without the top fully bonded layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))

In [18]:
# Adding top layers
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10, activation='softmax'))

In [19]:
# Freezing scales model VGG16
base_model.trainable = False

In [21]:
# Model compilation
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Model training
history = model.fit(train_images_preprocessed, train_labels, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
750/750 [==============================] - 1433s 2s/step - loss: 1.3310 - accuracy: 0.7348 - val_loss: 0.4917 - val_accuracy: 0.8211
Epoch 2/5
750/750 [==============================] - 1379s 2s/step - loss: 0.5418 - accuracy: 0.8102 - val_loss: 0.4320 - val_accuracy: 0.8441
Epoch 3/5
750/750 [==============================] - 1424s 2s/step - loss: 0.4809 - accuracy: 0.8260 - val_loss: 0.4278 - val_accuracy: 0.8472
Epoch 4/5
750/750 [==============================] - 1442s 2s/step - loss: 0.4496 - accuracy: 0.8365 - val_loss: 0.4153 - val_accuracy: 0.8512
Epoch 5/5
750/750 [==============================] - ETA: 0s - loss: 0.4282 - accuracy: 0.8431

In [24]:
# Evaluating accuracy on test data
test_loss, test_acc = model.evaluate(test_images_preprocessed, test_labels)
print('Test accuracy:', test_acc)

313/313 [==============================] - 232s 741ms/step - loss: 0.4249 - accuracy: 0.8515
Test accuracy: 0.8514999747276306
